In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
v0 = np.loadtxt('mpi0.volume.dat.gz', usecols=[1])
v1 = np.loadtxt('mpi1.volume.dat.gz', usecols=[1])
plt.plot(40/v0)
plt.plot(20/v1)
#plt.plot(v0+v1)

In [ ]:
v0